In [41]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas as pd

## Load Fakenews csv file

In [42]:
rootuser = 'admin'
your_pwd = '19980903'
path_input = '/Volumes/documents/news/data.csv'
name_Database = 'm2'

In [46]:
# Connect to PostgreSQL
con = psycopg2.connect(database="postgres", user=rootuser, password=your_pwd,host="127.0.0.1",port="5432")
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
# Create a cursor object using the cursor() method
cur = con.cursor()
# Setup a database with name wikinews
cur.execute("DROP DATABASE IF EXISTS "+name_Database+";")
cur.execute("CREATE DATABASE "+name_Database+";")
con.commit()
con.close()
create schema
con = psycopg2.connect(database=name_Database,user=rootuser,password=your_pwd,host='127.0.0.1',port='5432')
cur = con.cursor()
myschema = 'fakenews'
cur.execute("create schema " +myschema+";") 
#cur.execute("GRANT ALL PRIVILEGES ON SCHEMA " +myschema+ " TO GROUP xq;") 

con.commit()
con.close()
print("Database \'"+name_Database+"\' created successfully!")

Database 'm2' created successfully!


In [47]:
con = psycopg2.connect(database=name_Database,user='admin',password=your_pwd,host='127.0.0.1',port='5432')
cur = con.cursor()

In [48]:
# table name
name_table = myschema+'.news'

In [49]:
# Create a table called 'news' in the database
cur.execute('DROP TABLE IF EXISTS '+name_table+';')
cur.execute("CREATE TABLE "+name_table+"(index TEXT,id TEXT,domain TEXT,type TEXT,url TEXT,content TEXT,scraped_at DATE,inserted_at DATE,updated_at DATE,title TEXT,authors TEXT,keywords TEXT,meta_keywords TEXT,meta_description TEXT,tags TEXT,summary TEXT,source TEXT);")

# Copy data from csv file to pSQL this database m2
cur.execute("COPY "+name_table+" FROM \'"+path_input+"\' DELIMITER ',' CSV HEADER;") # Or CSV HEADER ENCODING 'UTF8';

# Delete news without 'id' attribute
cur.execute('DELETE FROM '+name_table+' WHERE id IS NULL')
# Delete text in 'id' attribute
cur.execute("DELETE FROM "+name_table+" WHERE id ~* '^[a-zA-Z]|\d*\.\d*'")
# Set integer for id
cur.execute("ALTER TABLE "+name_table+" ALTER COLUMN id TYPE integer USING id::integer") 
con.commit()
print("Database \'"+name_Database+"\' table \'"+name_table+"\' successfully filled in data from \'"+path_input+"\'!")

Database 'm2' table 'fakenews.news' successfully filled in data from '/Volumes/documents/news/data.csv'!


In [50]:
# Remove duplicate news
cur.execute('DELETE FROM '+name_table+' n WHERE n.id = ANY(ARRAY (SELECT id FROM (SELECT row_number() OVER (PARTITION BY authors,domain,title,content,meta_description,meta_keywords,tags), id FROM '+name_table+') t WHERE t.row_number > 1))')

In [51]:
con.commit()

In [58]:
# Connect to pSQL
con = psycopg2.connect(database=name_Database,user='admin',password=your_pwd,host='127.0.0.1',port='5432')
cur = con.cursor()

### **1.1** Creat table `domain`

In [53]:
# create table domain
cur.execute("DROP TABLE IF EXISTS fakenews.domain")
cur.execute("SELECT id,domain INTO fakenews.domain FROM "+name_table+";")
# delete duplicate domains
cur.execute('DELETE FROM fakenews.domain d WHERE d.id NOT IN (SELECT MIN(id) FROM fakenews.domain GROUP BY domain)')
# drop id
cur.execute('ALTER TABLE fakenews.domain DROP COLUMN id')
# generate domain_id
cur.execute('ALTER TABLE fakenews.domain ADD COLUMN d_id SERIAL UNIQUE')
# set domain as Primary key (PK)
cur.execute('ALTER TABLE fakenews.domain ADD PRIMARY KEY(d_id)')
con.commit()

### **1.2** Creat table `URL`

In [54]:
# create table URL
cur.execute("DROP TABLE IF EXISTS fakenews.URL")
cur.execute("SELECT id,URL,domain INTO fakenews.URL FROM "+name_table+";")
# rename id to url_id
cur.execute('ALTER TABLE fakenews.URL RENAME COLUMN id TO url_id')
# insert domain_id
cur.execute('ALTER TABLE fakenews.URL ADD column d_id integer')
cur.execute('UPDATE fakenews.URL SET d_id = fakenews.domain.d_id FROM fakenews.domain WHERE fakenews.URL.domain=fakenews.domain.domain')
cur.execute('ALTER TABLE fakenews.URL DROP COLUMN domain')
# set url_id as PK
cur.execute('ALTER TABLE fakenews.URL ADD PRIMARY KEY (url_id)')
con.commit()

### **1.3** Creat table `date`

In [55]:
# create table date
cur.execute("DROP TABLE IF EXISTS fakenews.date")
cur.execute("SELECT id,scraped_at,inserted_at,updated_at INTO fakenews.date FROM "+name_table+";")
# delete duplicate date
cur.execute('DELETE FROM fakenews.date d WHERE d.id NOT IN (SELECT MIN(id) FROM fakenews.date GROUP BY scraped_at,inserted_at,updated_at)')
# add date_id
cur.execute('ALTER TABLE fakenews.date DROP COLUMN id')
cur.execute('CREATE SEQUENCE fakenews.date_id')
cur.execute("ALTER TABLE fakenews.date ADD COLUMN date_id integer NOT NULL DEFAULT NEXTVAL('fakenews.date_id')")
cur.execute('ALTER SEQUENCE fakenews.date_id OWNED BY fakenews.date.date_id')
# set date_id as PK
cur.execute('ALTER TABLE fakenews.date ADD PRIMARY KEY (date_id)')
con.commit()

### **1.4** Creat table `article`

In [56]:
# create table article
cur.execute("DROP TABLE IF EXISTS fakenews.article")
cur.execute("SELECT id,title,summary,content,scraped_at,inserted_at,updated_at,meta_keywords,meta_description,tags INTO fakenews.article FROM "+name_table+";")
# insert date_id
cur.execute('ALTER TABLE fakenews.article ADD column date_id integer')
cur.execute('UPDATE fakenews.article SET date_id = fakenews.date.date_id FROM fakenews.date WHERE fakenews.article.scraped_at=fakenews.date.scraped_at AND fakenews.article.inserted_at=fakenews.date.inserted_at AND fakenews.article.updated_at=fakenews.date.updated_at')
cur.execute('ALTER TABLE fakenews.article DROP COLUMN scraped_at, DROP COLUMN inserted_at, DROP COLUMN updated_at')
# set id as PK
cur.execute('ALTER TABLE fakenews.article ADD PRIMARY KEY (id)')
# set date_id as foreign key (FK)
cur.execute('ALTER TABLE fakenews.article ADD CONSTRAINT fk_date FOREIGN KEY (date_id) REFERENCES fakenews.date(date_id)')
con.commit()

### **1.5** Creat table `from_where`

In [59]:
# create table from_where
cur.execute("DROP TABLE IF EXISTS fakenews.from_where")
cur.execute("SELECT id INTO fakenews.from_where FROM "+name_table+";")
cur.execute('ALTER TABLE fakenews.from_where ADD COLUMN url_id integer')
cur.execute('UPDATE fakenews.from_where SET url_id=id;')
# set FK
cur.execute('ALTER TABLE fakenews.from_where ADD CONSTRAINT fk_fromurlid FOREIGN KEY (url_id) REFERENCES fakenews.URL(url_id)')
cur.execute('ALTER TABLE fakenews.from_where ADD CONSTRAINT fk_fromartid FOREIGN KEY (id) REFERENCES fakenews.article(id)')
con.commit()

### **1.6** Creat table `type`

In [60]:
# create table type
cur.execute("DROP TABLE IF EXISTS fakenews.type")
cur.execute("SELECT id,type into fakenews.type from "+name_table+";")
# delete duplicate type
cur.execute('DELETE FROM fakenews.type t WHERE t.id NOT IN (SELECT MIN(id) FROM fakenews.type GROUP BY type)')
# add date_id
cur.execute('ALTER TABLE fakenews.type DROP COLUMN id')
cur.execute('CREATE SEQUENCE fakenews.type_id')
cur.execute("ALTER TABLE fakenews.type ADD COLUMN type_id integer NOT NULL DEFAULT NEXTVAL('fakenews.type_id')")
cur.execute('ALTER SEQUENCE fakenews.type_id OWNED BY fakenews.type.type_id')
# set date_id as PK
cur.execute('ALTER TABLE fakenews.type ADD PRIMARY KEY (type_id)')
con.commit()

### **1.7** Creat table `aut`

In [61]:
# create table aut(author)
cur.execute("DROP TABLE IF EXISTS fakenews.aut")
cur.execute("SELECT id,authors INTO fakenews.aut FROM "+name_table+";")
# delete dup authors
cur.execute('DELETE FROM fakenews.aut a WHERE a.id NOT IN (SELECT MIN(id) FROM fakenews.aut GROUP BY authors)')
# drop id
cur.execute('ALTER TABLE fakenews.aut DROP COLUMN id')
# gen aut_id
cur.execute('ALTER TABLE fakenews.aut add COLUMN aut_id SERIAL UNIQUE')
# set domain as PK
cur.execute('ALTER TABLE fakenews.aut ADD PRIMARY KEY(aut_id)')
con.commit()

### **1.8** Creat table `w_by`

In [62]:
# create table from_where
cur.execute("DROP TABLE IF EXISTS fakenews.w_by")
cur.execute("SELECT id,authors INTO fakenews.w_by FROM "+name_table+";")
# insert aut_id
cur.execute('ALTER TABLE fakenews.w_by ADD COLUMN aut_id integer')
cur.execute('UPDATE fakenews.w_by SET aut_id = fakenews.aut.aut_id FROM fakenews.aut WHERE fakenews.aut.authors=fakenews.w_by.authors')
cur.execute('ALTER TABLE fakenews.w_by DROP COLUMN authors')
# set FK
cur.execute('ALTER TABLE fakenews.w_by ADD CONSTRAINT fk_autid FOREIGN KEY (aut_id) REFERENCES fakenews.aut(aut_id)')
cur.execute('ALTER TABLE fakenews.w_by ADD CONSTRAINT fk_artautid FOREIGN KEY (id) REFERENCES fakenews.article(id)')
con.commit()

### **1.9** Creat table `is_type`

In [63]:
# create table from_where
cur.execute("DROP TABLE IF EXISTS fakenews.is_type")
cur.execute("SELECT id,type INTO fakenews.is_type FROM "+name_table+";")
# insert type_id
cur.execute('ALTER TABLE fakenews.is_type ADD COLUMN type_id integer')
cur.execute('UPDATE fakenews.is_type SET type_id=fakenews.type.type_id FROM fakenews.type WHERE fakenews.is_type.type=fakenews.type.type')
cur.execute('ALTER TABLE fakenews.is_type DROP COLUMN type')
# set FK
cur.execute('ALTER TABLE fakenews.is_type ADD CONSTRAINT fk_typeid FOREIGN KEY (type_id) REFERENCES fakenews.type(type_id)')
cur.execute('ALTER TABLE fakenews.is_type ADD CONSTRAINT fk_arttypeid FOREIGN KEY (id) REFERENCES fakenews.article(id)')
con.commit()